NOTE:
This is a directly exported ipynb version of the databricks notebook. 
 To check out the .dbc version, visit 
this [link](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/5765572329082963/819669162567595/1062157151406693/latest.html)

Import required libraries

In [0]:
from pyspark.sql.functions import col, udf, regexp_replace, isnull
from pyspark.sql.types import StringType,IntegerType

from pyspark.ml.feature import StringIndexer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF

from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

Load the data

In [0]:
# File location and type
file_location = "/FileStore/tables/newsCorpora.csv"

df = spark.read.csv(file_location, 
                    header=False,
                    sep='\t')

In [0]:
df.show()

+---+--------------------+--------------------+--------------------+---+--------------------+--------------------+-------------+
_c0| _c1| _c2| _c3|_c4| _c5| _c6| _c7|
+---+--------------------+--------------------+--------------------+---+--------------------+--------------------+-------------+
 1|Fed official says...|http://www.latime...| Los Angeles Times| b|ddUyU0VZz0BRneMio...| www.latimes.com|1394470370698|
 2|Fed's Charles Plo...|http://www.livemi...| Livemint| b|ddUyU0VZz0BRneMio...| www.livemint.com|1394470371207|
 3|US open: Stocks f...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371550|
 4|Fed risks falling...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...| www.ifamagazine.com|1394470371793|
 5|Fed's Plosser: Na...|http://www.moneyn...| Moneynews| b|ddUyU0VZz0BRneMio...| www.moneynews.com|1394470372027|
 6|Plosser: Fed May ...|http://www.nasdaq...| NASDAQ| b|ddUyU0VZz0BRneMio...| www.nasdaq.com|1394470372212|
 7|Fed's Plosser: Ta...|http://www.market...| MarketWatch| b|ddUyU0VZz0BRneMio...| www.marketwatch.com|1394470372405|
 8|Fed's Plosser exp...|http://www.fxstre...| FXstreet.com| b|ddUyU0VZz0BRneMio...| www.fxstreet.com|1394470372615|
 9|US jobs growth la...|http://economicti...| Economic Times| b|ddUyU0VZz0BRneMio...|economictimes.ind...|1394470372792|
 10|ECB unlikely to e...|http://www.iii.co...|Interactive Investor| b|dPhGU51DcrolUIMxb...| www.iii.co.uk|1394470501265|
 11|ECB unlikely to e...|http://in.reuters...| Reuters India| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501410|
 12|EU's half-baked b...|http://blogs.reut...| Reuters UK \(blog\)| b|dPhGU51DcrolUIMxb...| blogs.reuters.com|1394470501587|
 13|Europe reaches cr...|http://in.reuters...| Reuters| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501755|
 14|ECB FOCUS-Stronge...|http://in.reuters...| Reuters| b|dPhGU51DcrolUIMxb...| in.reuters.com|1394470501948|
 15|EU aims for deal ...|http://main.omano...| Oman Daily Observer| b|dPhGU51DcrolUIMxb...|main.omanobserver.om|1394470502141|
 16|Forex - Pound dro...|http://www.nasdaq...| NASDAQ| b|dPhGU51DcrolUIMxb...| www.nasdaq.com|1394470502316|
 17|Noyer Says Strong...|http://www.sfgate...|San Francisco Chr...| b|dPhGU51DcrolUIMxb...| www.sfgate.com|1394470502543|
 18|EU Week Ahead Mar...|http://blogs.wsj....|Wall Street Journ...| b|dPhGU51DcrolUIMxb...| blogs.wsj.com|1394470502744|
 19|ECB member Noyer ...|http://www.ifamag...| IFA Magazine| b|dPhGU51DcrolUIMxb...| www.ifamagazine.com|1394470502946|
 20|Euro Anxieties Wa...|http://www.busine...| Businessweek| b|dPhGU51DcrolUIMxb...|www.businessweek.com|1394470503148|
+---+--------------------+--------------------+--------------------+---+--------------------+--------------------+-------------+
only showing top 20 rows

Renaming the columns

In [0]:
old_cols = df.schema.names
new_cols = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']

from functools import reduce
df = reduce(lambda df, idx: df.withColumnRenamed(old_cols[idx], new_cols[idx]), range(len(old_cols)), df)

In [0]:
df.show(5)

+---+--------------------+--------------------+-----------------+--------+--------------------+-------------------+-------------+
 ID| TITLE| URL| PUBLISHER|CATEGORY| STORY| HOSTNAME| TIMESTAMP|
+---+--------------------+--------------------+-----------------+--------+--------------------+-------------------+-------------+
 1|Fed official says...|http://www.latime...|Los Angeles Times| b|ddUyU0VZz0BRneMio...| www.latimes.com|1394470370698|
 2|Fed's Charles Plo...|http://www.livemi...| Livemint| b|ddUyU0VZz0BRneMio...| www.livemint.com|1394470371207|
 3|US open: Stocks f...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...|www.ifamagazine.com|1394470371550|
 4|Fed risks falling...|http://www.ifamag...| IFA Magazine| b|ddUyU0VZz0BRneMio...|www.ifamagazine.com|1394470371793|
 5|Fed's Plosser: Na...|http://www.moneyn...| Moneynews| b|ddUyU0VZz0BRneMio...| www.moneynews.com|1394470372027|
+---+--------------------+--------------------+-----------------+--------+--------------------+-------------------+-------------+
only showing top 5 rows

In [0]:
print(f'Total number of rows: {df.count()}')

Total number of rows: 422937

Subsetting a new dataframe with dependent variable (category) and independent variable (title)

In [0]:
news_df = df.select("TITLE", "CATEGORY")

Missing values

In [0]:
news_df.where(col("TITLE").isNull()).show()

+-----+--------+
TITLE|CATEGORY|
+-----+--------+
+-----+--------+

In [0]:
news_df.where(col("CATEGORY").isNull()).show()

+--------------------+--------+
 TITLE|CATEGORY|
+--------------------+--------+
Love & Hip-Hop' S...| null|
Peaches has died....| null|
I don't know why ...| null|
Hunger Games' top...| null|
A few years ago, ...| null|
I never thought I...| null|
The Best Reaction...| null|
The Fault In Our ...| null|
More Japanese Xbo...| null|
+--------------------+--------+

In [0]:
news_df.groupBy('CATEGORY').count().orderBy(col('count').desc()).show()

+--------+------+
CATEGORY| count|
+--------+------+
 e|152821|
 b|115967|
 t|108501|
 m| 45639|
 null| 9|
+--------+------+

In [0]:
news_data = news_df.where(col("CATEGORY").isNotNull())

In [0]:
null_values = news_df.where(col("CATEGORY").isNull())

In [0]:
null_values.select('TITLE').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
TITLE |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
Love & Hip-Hop' Star Benzino Shot By Nephew En Route To Mother's Funeral	http://www.designntrend.com/articles/12221/20140330/love-hip-hop-star-benzino-shot-nephew-en-route-mothers.htm	Design \& Trend	e	dMXJCPWNIRd9VWMrvxGgoEIc8UaMM	www.designntrend.com	1396236920861

Peaches has died. We are beyond pain.	http://www.itv.com/news/2014-04-07/bob-geldof-full-statement-peaches-was-the-most-bonkers-of-us-all/	ITV News	e	dVcLyu7MZ2lvOuM8psiqi0ay7A2GM	www.itv.com	1396997444213

I don't know why I'm a sex symbol' says smouldering hunk of manliness Jon ...	http://www.irishexaminer.com/breakingnews/entertainment/i-dont-know-why-im-a-sex-symbol-says-smouldering-hunk-of-manliness-jon-hamm-627395.html	Irish Examiner	e	dQp_0Hv3qTdoGMMXikOnSJKtYXH-M	www.irishexaminer.com	1397237964614

Hunger Games' top winner at MTV Movie Awards	http://www.tuscaloosanews.com/article/20140413/NEWS/140419856/1291/news\?Title=-Hunger-Games-8217-top-winner-at-MTV-Movie-Awards-	Tuscaloosa Magazine	e	dZzicVR_bvCJYFMk1LFUOBLlD4i9M	www.tuscaloosanews.com	1397459817729

A few years ago, scientists calculated that benefit as having a net present value ...	http://theenergycollective.com/josephromm/367921/climate-panel-stunner-avoiding-climate-catastrophe-super-cheap-only-if-we-act-now	Energy Collective	t	dPhzj-Zymt7_2fMuSkTFhziYo3U0M	theenergycollective.com	1397513703600

I never thought I'd be in love” says Angelina Jolie	http://www.shanghaidaily.com/article/article_xinhua.aspx\?id=217113	Shanghai Daily \(subscription\)	e	dSK9w1sX3_u4D4MgvroyRJ-CcxxAM	www.shanghaidaily.com	1399574518790

The Best Reactions To The Supposed Video of Solange Knowles & Jay Z ...	http://www.hiphopdx.com/index/news/id.28728/title.-the-best-reactions-to-the-supposed-video-of-solange-knowles-jay-z-fighting-in-an-elevator-list-by-time/	HipHopDX	e	dku0uRoeehpC9JM1RoZ4n0fg8cyoM	www.hiphopdx.com	1399983366398

The Fault In Our Stars' to release in India on July 4	http://www.saharasamay.com/entertainment-news/676554828/hollywood-blockbuster-the-fault-in-our-stars-to-release-in-india.html	Sahara Samay	e	d6gvNZSb4XfZc4Mu_LM_scaSHr47M	www.saharasamay.com	1402677197466

More Japanese Xbox One Games In Development, Applications for ID@Xbox ...	http://www.gamepur.com/news/15488-more-japanese-xbox-one-games-development-applications-idxbox-are-every.html	Gamepur	t	dkWNcdUBfwURMjMn2vv2oJJy4k58M	www.gamepur.com	1405708685335

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

We can see, it is not actually Null, our csv reader was not able to split the rows properly. Thus, we go ahead and split the title into required columns.

In [0]:
null_values_pd = null_values.toPandas()


In [0]:
null_values_pd.TITLE.values[0]

Out[210]: "Love & Hip-Hop' Star Benzino Shot By Nephew En Route To Mother's Funeral\thttp://www.designntrend.com/articles/12221/20140330/love-hip-hop-star-benzino-shot-nephew-en-route-mothers.htm\tDesign \\& Trend\te\tdMXJCPWNIRd9VWMrvxGgoEIc8UaMM\twww.designntrend.com\t1396236920861\n"

In [0]:
null_values_pd.TITLE.values[0].split('\t')

Out[211]: ["Love & Hip-Hop' Star Benzino Shot By Nephew En Route To Mother's Funeral",
 'http://www.designntrend.com/articles/12221/20140330/love-hip-hop-star-benzino-shot-nephew-en-route-mothers.htm',
 'Design \\& Trend',
 'e',
 'dMXJCPWNIRd9VWMrvxGgoEIc8UaMM',
 'www.designntrend.com',
 '1396236920861\n']

In [0]:
categories = [null_values_pd.TITLE.values[i].split('\t')[3] for i in range(len(null_values_pd))]
categories

Out[216]: ['e', 'e', 'e', 'e', 't', 'e', 'e', 'e', 't']

In [0]:
null_values_pd.loc[:, 'CATEGORY'] = categories

In [0]:
null_values_pd

,TITLE,CATEGORY
0,Love & Hip-Hop' Star Benzino Shot By Nephew En...,e
1,Peaches has died. We are beyond pain.\thttp://...,e
2,I don't know why I'm a sex symbol' says smould...,e
3,Hunger Games' top winner at MTV Movie Awards\t...,e
4,"A few years ago, scientists calculated that be...",t
5,I never thought I'd be in love” says Angelina ...,e
6,The Best Reactions To The Supposed Video of So...,e
7,The Fault In Our Stars' to release in India on...,e
8,"More Japanese Xbox One Games In Development, A...",t


In [0]:
imputed_sdf = spark.createDataFrame(null_values_pd)
imputed_sdf.show()

+--------------------+--------+
 TITLE|CATEGORY|
+--------------------+--------+
Love & Hip-Hop' S...| e|
Peaches has died....| e|
I don't know why ...| e|
Hunger Games' top...| e|
A few years ago, ...| t|
I never thought I...| e|
The Best Reaction...| e|
The Fault In Our ...| e|
More Japanese Xbo...| t|
+--------------------+--------+

Merging the DataFrames

In [0]:
news_data_imp = news_data.union(imputed_sdf)
news_data_imp.groupBy('CATEGORY').count().orderBy(col('count').desc()).show()

+--------+------+
CATEGORY| count|
+--------+------+
 e|152828|
 b|115967|
 t|108503|
 m| 45639|
+--------+------+

In [0]:
news_data_imp.select('TITLE').show(10, truncate=False)

+------------------------------------------------------------------------+
TITLE |
+------------------------------------------------------------------------+
Fed official says weak data caused by weather, should not slow taper |
Fed's Charles Plosser sees high bar for change in pace of tapering |
US open: Stocks fall after Fed official hints at accelerated tapering |
Fed risks falling 'behind the curve', Charles Plosser says |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |
Plosser: Fed May Have to Accelerate Tapering Pace |
Fed's Plosser: Taper pace may be too slow |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014|
US jobs growth last month hit by weather:Fed President Charles Plosser |
ECB unlikely to end sterilisation of SMP purchases - traders |
+------------------------------------------------------------------------+
only showing top 10 rows

Data Cleaning
- Regex
- Tokenizing
- Removing stop words

In [0]:
news_data_imp = news_data_imp.withColumn('clean_title', regexp_replace(col('TITLE'), r'\'s'  , ''))
news_data_imp = news_data_imp.withColumn('clean_title', regexp_replace(col('clean_title'), '[^a-zA-Z ]'  , ''))
news_data_imp.show(truncate=False)

+---------------------------------------------------------------------------+--------+---------------------------------------------------------------------+
TITLE |CATEGORY|clean_title |
+---------------------------------------------------------------------------+--------+---------------------------------------------------------------------+
Fed official says weak data caused by weather, should not slow taper |b |Fed official says weak data caused by weather should not slow taper |
Fed's Charles Plosser sees high bar for change in pace of tapering |b |Fed Charles Plosser sees high bar for change in pace of tapering |
US open: Stocks fall after Fed official hints at accelerated tapering |b |US open Stocks fall after Fed official hints at accelerated tapering |
Fed risks falling 'behind the curve', Charles Plosser says |b |Fed risks falling behind the curve Charles Plosser says |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |b |Fed Plosser Nasty Weather Has Curbed Job Growth |
Plosser: Fed May Have to Accelerate Tapering Pace |b |Plosser Fed May Have to Accelerate Tapering Pace |
Fed's Plosser: Taper pace may be too slow |b |Fed Plosser Taper pace may be too slow |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014 |b |Fed Plosser expects US unemployment to fall to by the end of |
US jobs growth last month hit by weather:Fed President Charles Plosser |b |US jobs growth last month hit by weatherFed President Charles Plosser|
ECB unlikely to end sterilisation of SMP purchases - traders |b |ECB unlikely to end sterilisation of SMP purchases traders |
ECB unlikely to end sterilization of SMP purchases: traders |b |ECB unlikely to end sterilization of SMP purchases traders |
EU's half-baked bank union could work |b |EU halfbaked bank union could work |
Europe reaches crunch point on banking union |b |Europe reaches crunch point on banking union |
ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|b |ECB FOCUSStronger euro drowns out ECB message to keep rates low for |
EU aims for deal on tackling failing banks |b |EU aims for deal on tackling failing banks |
Forex - Pound drops to one-month lows against euro |b |Forex Pound drops to onemonth lows against euro |
Noyer Says Strong Euro Creates Unwarranted Economic Pressure |b |Noyer Says Strong Euro Creates Unwarranted Economic Pressure |
EU Week Ahead March 10-14: Bank Resolution, Transparency, Ukraine |b |EU Week Ahead March Bank Resolution Transparency Ukraine |
ECB member Noyer is 'very open to all kinds of tools' |b |ECB member Noyer is very open to all kinds of tools |
Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |b |Euro Anxieties Wane as Bunds Top Treasuries Spain Debt Rallies |
+---------------------------------------------------------------------------+--------+---------------------------------------------------------------------+
only showing top 20 rows

In [0]:
re_tokenizer = RegexTokenizer(inputCol='clean_title', outputCol='tokens')
tokenized_data = re_tokenizer.transform(news_data_imp)

In [0]:
tokenized_data.select('TITLE','tokens').show(truncate=False)

+---------------------------------------------------------------------------+---------------------------------------------------------------------------------+
TITLE |tokens |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------------+
Fed official says weak data caused by weather, should not slow taper |[fed, official, says, weak, data, caused, by, weather, should, not, slow, taper] |
Fed's Charles Plosser sees high bar for change in pace of tapering |[fed, charles, plosser, sees, high, bar, for, change, in, pace, of, tapering] |
US open: Stocks fall after Fed official hints at accelerated tapering |[us, open, stocks, fall, after, fed, official, hints, at, accelerated, tapering] |
Fed risks falling 'behind the curve', Charles Plosser says |[fed, risks, falling, behind, the, curve, charles, plosser, says] |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |[fed, plosser, nasty, weather, has, curbed, job, growth] |
Plosser: Fed May Have to Accelerate Tapering Pace |[plosser, fed, may, have, to, accelerate, tapering, pace] |
Fed's Plosser: Taper pace may be too slow |[fed, plosser, taper, pace, may, be, too, slow] |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014 |[fed, plosser, expects, us, unemployment, to, fall, to, by, the, end, of] |
US jobs growth last month hit by weather:Fed President Charles Plosser |[us, jobs, growth, last, month, hit, by, weatherfed, president, charles, plosser]|
ECB unlikely to end sterilisation of SMP purchases - traders |[ecb, unlikely, to, end, sterilisation, of, smp, purchases, traders] |
ECB unlikely to end sterilization of SMP purchases: traders |[ecb, unlikely, to, end, sterilization, of, smp, purchases, traders] |
EU's half-baked bank union could work |[eu, halfbaked, bank, union, could, work] |
Europe reaches crunch point on banking union |[europe, reaches, crunch, point, on, banking, union] |
ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|[ecb, focusstronger, euro, drowns, out, ecb, message, to, keep, rates, low, for] |
EU aims for deal on tackling failing banks |[eu, aims, for, deal, on, tackling, failing, banks] |
Forex - Pound drops to one-month lows against euro |[forex, pound, drops, to, onemonth, lows, against, euro] |
Noyer Says Strong Euro Creates Unwarranted Economic Pressure |[noyer, says, strong, euro, creates, unwarranted, economic, pressure] |
EU Week Ahead March 10-14: Bank Resolution, Transparency, Ukraine |[eu, week, ahead, march, bank, resolution, transparency, ukraine] |
ECB member Noyer is 'very open to all kinds of tools' |[ecb, member, noyer, is, very, open, to, all, kinds, of, tools] |
Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |[euro, anxieties, wane, as, bunds, top, treasuries, spain, debt, rallies] |
+---------------------------------------------------------------------------+---------------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
sw_remover = StopWordsRemover(inputCol='tokens', outputCol='no_stop_words')
sw_tokenized_data = sw_remover.transform(tokenized_data)

In [0]:
sw_tokenized_data.select('TITLE', 'no_stop_words').show(truncate=False)

+---------------------------------------------------------------------------+-----------------------------------------------------------------------------+
TITLE |no_stop_words |
+---------------------------------------------------------------------------+-----------------------------------------------------------------------------+
Fed official says weak data caused by weather, should not slow taper |[fed, official, says, weak, data, caused, weather, slow, taper] |
Fed's Charles Plosser sees high bar for change in pace of tapering |[fed, charles, plosser, sees, high, bar, change, pace, tapering] |
US open: Stocks fall after Fed official hints at accelerated tapering |[us, open, stocks, fall, fed, official, hints, accelerated, tapering] |
Fed risks falling 'behind the curve', Charles Plosser says |[fed, risks, falling, behind, curve, charles, plosser, says] |
Fed's Plosser: Nasty Weather Has Curbed Job Growth |[fed, plosser, nasty, weather, curbed, job, growth] |
Plosser: Fed May Have to Accelerate Tapering Pace |[plosser, fed, may, accelerate, tapering, pace] |
Fed's Plosser: Taper pace may be too slow |[fed, plosser, taper, pace, may, slow] |
Fed's Plosser expects US unemployment to fall to 6.2% by the end of 2014 |[fed, plosser, expects, us, unemployment, fall, end] |
US jobs growth last month hit by weather:Fed President Charles Plosser |[us, jobs, growth, last, month, hit, weatherfed, president, charles, plosser]|
ECB unlikely to end sterilisation of SMP purchases - traders |[ecb, unlikely, end, sterilisation, smp, purchases, traders] |
ECB unlikely to end sterilization of SMP purchases: traders |[ecb, unlikely, end, sterilization, smp, purchases, traders] |
EU's half-baked bank union could work |[eu, halfbaked, bank, union, work] |
Europe reaches crunch point on banking union |[europe, reaches, crunch, point, banking, union] |
ECB FOCUS-Stronger euro drowns out ECB's message to keep rates low for ...|[ecb, focusstronger, euro, drowns, ecb, message, keep, rates, low] |
EU aims for deal on tackling failing banks |[eu, aims, deal, tackling, failing, banks] |
Forex - Pound drops to one-month lows against euro |[forex, pound, drops, onemonth, lows, euro] |
Noyer Says Strong Euro Creates Unwarranted Economic Pressure |[noyer, says, strong, euro, creates, unwarranted, economic, pressure] |
EU Week Ahead March 10-14: Bank Resolution, Transparency, Ukraine |[eu, week, ahead, march, bank, resolution, transparency, ukraine] |
ECB member Noyer is 'very open to all kinds of tools' |[ecb, member, noyer, open, kinds, tools] |
Euro Anxieties Wane as Bunds Top Treasuries, Spain Debt Rallies |[euro, anxieties, wane, bunds, top, treasuries, spain, debt, rallies] |
+---------------------------------------------------------------------------+-----------------------------------------------------------------------------+
only showing top 20 rows

In [0]:
indexer = StringIndexer(inputCol="CATEGORY", outputCol="category_index")
indexed_data = indexer.fit(sw_tokenized_data).transform(sw_tokenized_data)

In [0]:
indexed_data.groupBy('CATEGORY', 'category_index').count().orderBy(col('count').desc()).show()

+--------+--------------+------+
CATEGORY|category_index| count|
+--------+--------------+------+
 e| 0.0|152828|
 b| 1.0|115967|
 t| 2.0|108503|
 m| 3.0| 45639|
+--------+--------------+------+

#### Vectorizing the text data

Count Vectorizer

In [0]:
cv = CountVectorizer(inputCol='no_stop_words', outputCol='count_vec_features')
cv_model = cv.fit(indexed_data)
cv_features = cv_model.transform(indexed_data)

In [0]:
cv_features.select('count_vec_features').show(2)

+--------------------+
 count_vec_features|
+--------------------+
(60604,[5,36,104,...|
(60604,[78,82,104...|
+--------------------+
only showing top 2 rows

In [0]:
train_cv, test_cv = cv_features.randomSplit([0.75, 0.25],seed=9)

In [0]:
naive_bayes_cv = NaiveBayes(modelType='multinomial', labelCol='category_index', featuresCol='count_vec_features')
naive_bayes_cv_model = naive_bayes_cv.fit(train_cv)
predictions_cv = naive_bayes_cv_model.transform(test_cv)

In [0]:
model_evaluator_cv = MulticlassClassificationEvaluator(labelCol='category_index', predictionCol='prediction', metricName='accuracy')
accuracy_cv = model_evaluator_cv.evaluate(predictions_cv)

In [0]:
print(f"Accuracy using NB Classifier for count vectorized features = {round(accuracy_cv * 100, 3)}%")


Accuracy using NB Classifier for count vectorized features = 92.769%

TF-IDF Vectorizer

In [0]:
hashingTF = HashingTF(inputCol='no_stop_words', outputCol='tf_features')
tf_features_data = hashingTF.transform(indexed_data)

In [0]:
idf = IDF(inputCol="tf_features", outputCol="tfidf_features")
idfModel = idf.fit(tf_features_data)
tfidf_features_data = idfModel.transform(tf_features_data)

In [0]:
train_tfidf, test_tfidf = tfidf_features_data.randomSplit([0.75, 0.25], seed=9)

In [0]:
naive_bayes_tfidf = NaiveBayes(modelType="multinomial", labelCol="category_index", featuresCol="tfidf_features")
naive_bayes_tfidf_model = naive_bayes_tfidf.fit(train_tfidf)
predictions_tfidf = naive_bayes_tfidf_model.transform(test_tfidf)

In [0]:
model_evaluator_tfidf = MulticlassClassificationEvaluator(labelCol="category_index", predictionCol="prediction", metricName="accuracy")
accuracy_tfidf = model_evaluator_tfidf.evaluate(predictions_tfidf)

In [0]:
print(f"Accuracy using NB Classifier for tf-idf vectorized features = {round(accuracy_tfidf * 100, 3)}%")

Accuracy using NB Classifier for tf-idf vectorized features = 92.783%